In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from os import path

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

## Load data
Load interpolated/point data for vessels/whales - this is at 10 minute resolution, allowing detection of encounters between vessels/whales

In [ ]:
from data_processing import load_data

In [ ]:
whales, vessels, protected_areas, basemap, bounds = load_data.load_all()

In [ ]:
vessel_data_files = [path.join('data', 'vessels', f'{vessel_type}_points.gpkg') for vessel_type in ['Fishing', 'Other', 'Cargo', 'Passenger', 'Tanker']]
vessel_data_sets = [load_data.load_vessel_points(vdf, 2193) for vdf in vessel_data_files]

vessel_points = pd.concat(vessel_data_sets)

In [ ]:
whales_points = load_data.load_whales('data/whales/df_all_3.csv', vessel_points.geometry.total_bounds, 2193, 10)

## Plotting
Plot vessel + whale traces, color encounters

In [ ]:
from plotting import heatmap

In [ ]:
from bokeh.io import output_notebook, export_png
from bokeh.plotting import show, figure
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter

output_notebook()

## Create animation frames

In [ ]:
start = '2022-08-01'
end = '2022-09-01'
whale_mask = (whales_points.timestamp >= start) & (whales_points.timestamp < end)
vessel_mask = (vessel_points.timestamp >= start) & (vessel_points.timestamp < end)
timestamps = pd.date_range(start, end, freq='30min')

In [ ]:
for ind, ts in enumerate(timestamps[::5]):
    print(ind)
    fig = heatmap.animation_frame(whales_points[whale_mask], vessel_points[vessel_mask], protected_areas, basemap, bounds, ts)
    export_png(fig, filename=f'frames/{ind}.png')

## Incomplete work on a streaming version below
This could be useful for embedding this animation as a static webpage with slider, which would require:
- A column data source containing all vessels, +1 for whales
- CustomJS on a slider to replace the actual CDS's (1 per vessel/whale) with slices from the complete source CDS

In [ ]:
# Callsign/id/timestamp indices
callsigns = vessel_points[vessel_mask].callsign.unique()
wh_ids = whales_points[whale_mask].id.unique()
timestamps = pd.date_range(start, end, freq='30min')
timestamps_int = timestamps.astype('int64') / 1e9

# Build column x/y per callsign/id
vessel_cols = {'timestamp': timestamps_int}
for callsign, group in vessel_points[vessel_mask].groupby('callsign'):
    group = group.set_index('timestamp').reindex(timestamps, method='nearest').reset_index(
        names='timestamp')
    vessel_cols[f'{callsign}_x'] = group.geometry.x
    vessel_cols[f'{callsign}_y'] = group.geometry.y

whale_cols = {'timestamp': timestamps_int}
for wh_id, group in whales_points[whale_mask].groupby('id'):
    group = group.set_index('timestamp').reindex(timestamps, method='nearest').reset_index(
        names='timestamp')
    whale_cols[f'{wh_id}_x'] = group.geometry.x
    whale_cols[f'{wh_id}_y'] = group.geometry.y
    
# Build column source for vessels/whales
vessel_source = ColumnDataSource(vessel_cols)
whale_source = ColumnDataSource(whale_cols)

Note that the below doesn't work - you can't filter data for glyphs that require continuity

In [ ]:
# CDSView filtering up to current timestamp
vessel_view = CDSView(filter=BooleanFilter(vessel_cols['timestamp'] <= timestamps_int[1000]))
whale_view = CDSView(filter=BooleanFilter(whale_cols['timestamp'] <= timestamps_int[1000]))

In [ ]:
m = timestamps_int < timestamps_int[800]

ve_data = {k: v[m] for k, v in vessel_cols.items()}

In [ ]:
# Plot
fig = figure()
for cs in callsigns:
    fig.line(x=ve_data[f'{cs}_x'], y=ve_data[f'{cs}_y'], color='blue')
# fig.multi_line(xs=[f'{callsign}_x' for callsign in callsigns], ys=[f'{callsign}_y' for callsign in callsigns], source=vessel_source, color='blue')
# fig.multi_line(xs=[f'{wh_id}_x' for wh_id in wh_ids], ys=[f'{wh_id}_y' for wh_id in wh_ids], source=whale_source, view=whale_view, color='red')

show(fig)